In [2]:
import pandas as pd
import re

In [10]:
data = pd.read_csv('./data/eu_life_expectancy_raw.tsv', sep= '\,|\t', header=0, engine='python')
data = data.melt(id_vars=['unit', 'sex', 'age', 'geo\\time'])
data = data.rename(columns={'geo\\time' : 'region', 'variable': 'year'})
data.value = data.value.apply(lambda x: None if ':' in x else x).str.replace(r'[^0-9.]', '', regex=True)
data = data.astype({'year': int, 'value': float})
data = data.dropna()
data_only_pt = data[data['region'] == 'PT']
data_only_pt
# data_only_pt.to_csv('./data/pt_life_expectancy.csv', index=False)

,unit,sex,age,region,year,value
3460,YR,F,Y65,PT,2021,21.7
4804,YR,F,Y_LT1,PT,2021,84.3
8276,YR,M,Y65,PT,2021,17.8
9620,YR,M,Y_LT1,PT,2021,78.0
13092,YR,T,Y65,PT,2021,19.9
...,...,...,...,...,...,...
895540,YR,T,Y83,PT,1960,4.7
895596,YR,T,Y84,PT,1960,4.5
895652,YR,T,Y9,PT,1960,62.9
895708,YR,T,Y_GE85,PT,1960,4.2


In [6]:
data_only_pt.dtypes

unit       object
sex        object
age        object
region     object
year        int64
value     float64
dtype: object